In [0]:
import openreview
from collections import Counter, defaultdict as dd
import numpy as np
import pandas as pd
import qgrid
from pprint import pprint
import re 
import time
import jellyfish

In [0]:
def all_papers():
    client = openreview.Client(baseurl='https://openreview.net')
    paper_iterator = openreview.tools.iterget_notes(client, invitation='ICLR.cc/2019/Conference/-/Blind_Submission')
    return list(paper_iterator)

In [0]:
def ratings_and_confidence():
    client = openreview.Client(baseurl='https://openreview.net')
    review_iterator = openreview.tools.iterget_notes(client, invitation='ICLR.cc/2019/Conference/-/Paper.*/Official_Review')
    ratings = dd(list)
    confidence = dd(list)
    for review in review_iterator:
        ratings[review.forum].append(int(re.findall(pattern='\d+', string=review.content['rating'])[0]))
        confidence[review.forum].append(int(re.findall(pattern='\d+', string=review.content['confidence'])[0]))
    return ratings, confidence

In [0]:
papers = all_papers()
ratings, confidence = ratings_and_confidence()

In [0]:
def add_ratings(papers, ratings, confidence):
    for ix in range(len(papers)):
        forum = papers[ix].forum
        papers[ix].ratings = ratings.get(forum, [])
        papers[ix].confidence = confidence.get(forum, [])

In [0]:
add_ratings(papers, ratings, confidence)

In [0]:
def get_keywords(papers):
    return list(set([word.strip().lower() for p in papers for word in p.content['keywords']]))

def get_similarity(keywords):
    sim = dict()
    N = len(keywords)
    for i in range(N-1):
        for j in range(i+1, N):
            topic1 = keywords[i]
            topic2 = keywords[j]
            dist = len(set(topic1).intersection(set(topic2)))/len(set(max([topic1, topic2], key=lambda v: len(v))))
            if topic1 != topic2 and dist > 0.75:
                dist = jellyfish.levenshtein_distance(topic1, topic2)
                sim[topic1, topic2] = dist
    return sim

def get_top_matches(sim):
    matches = dd(list)
    for pair in sim:
        if sim[pair] == 1:
            matches[pair[0]].append(pair[1])
            matches[pair[1]].append(pair[0])
    return matches    

In [40]:
keywords = get_keywords(papers)
start = time.time()
sim = get_similarity(keywords) # this may take several minutes 
finish = time.time()
print('Similarity computed in {:.2f} sec'.format(finish - start))

Similarity computed in 17.09 sec


In [0]:
matches = get_top_matches(sim)

In [0]:
def get_clusters(matches):
    clusters = dict()
    for topic in matches:
        values = matches[topic]
        clusters[topic] = topic
        for v in values:
            if len(matches.get(v, [])) > len(values) or \
            (len(matches.get(v, [])) == len(values) and topic > v):
                clusters[topic] = v
    return clusters

In [0]:
clusters = get_clusters(matches)

In [0]:
def get_df(papers, clusters):
    rows = []
    for paper in papers:
        topic = paper.content['keywords'][0] if len(paper.content['keywords']) else ''
        cluster = clusters.get(topic.lower(), topic.lower() if topic else None)
        row = dict(tldr = paper.content.get('TL;DR', ''),
                  ratings = getattr(paper, 'ratings', []),
                  confidence = getattr(paper, 'confidence', []),
                  title = paper.content['title'],
                  avg_rating = np.mean(getattr(paper, 'ratings', [])),
                  avg_confidence = np.mean(getattr(paper, 'confidence', [])),
                  topic = cluster,
                  url = "https://openreview.net/forum?id=" + paper.forum
                  )
        rows.append(row)
    df = pd.DataFrame(rows)
    df.index.name = 'Index'
    df = df.iloc[:, [4,1,0,6,5,3,2,7]]
    return df 

In [0]:
df = get_df(papers, clusters)

In [0]:
col_defs = {
    name: {
        'width': 50
    } for name in ['Index', 'confidence', 'ratings']
}
col_defs['title'] = {'width': 500}
col_defs['avg_rating'] = {'width': 100}
col_defs['tldr'] = {'width': 1000}
col_defs['url'] = {'width': 300}
col_defs['topic'] = {'width': 200}
grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100}
grid = qgrid.show_grid(df, column_definitions=col_defs, grid_options = grid_options, precision=1,)

In [54]:
grid

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…